# Variances and Beliefs

Here I want to see whether pessimistic beliefs have an impact on the variance of the assets or not. 
<br\>
* First, I will write a function that returns a belief decompostition of a transition matrix, a real transition matrix and a corresponding belief matrix with pessimistic beliefs.
<br\>
* Then, I will write a class and corresponding functions, which distinguish between the two different kinds of transition matrices.
<br\>
* After that, I will simulate the asset choices given a interest rate, compute the variance of the assets series and compare them.
<br\>
* Finally, I will write a function which return a mean preserving spread of the state values given the belief matrix, then use that to compute another asset series which I can compare to the both before.



In [139]:
# First, I will write a function that returns a belief 
# decompostition of a transition matrix, a real transition 
# matrix and a corresponding belief matrix with pessimistic beliefs.

import numpy as np
from quantecon import MarkovChain

def trans_mat(P1, P3, epsilon):
    """
    Returns a transition matrix (for a markov chain) of the following form:
        
        Π = ((P1, P2, 0), (P2 - epsilon, P1, epsilon), (P4, P4, P3))
    
    where P2 = 1-P1 and P4 = 0.5*(1-P3)
    
    epsilon is the probability for the transition from the 2. to the 
    3. state.
    P1 is the probability to stay in the current state if the 
    current state is the 1. or 2. state.
    P3 ist the probability to stay in the 3rd state.
    2*P4 ist the probability to get out of the 3rd state.
    """
    P2, P4 = (1-P1), 0.5*(1-P3)
    
    Π = ((P1, P2, 0), (P2 - epsilon, P1, epsilon), (P4, P4, P3))
    
    return np.asarray(Π)


def belief_decompose(Π, z_vals):
    
    """
    Belief decomposition for a 3x3 matrix with the follwowing form:
        
        Π = ((P1, P2, 0), (P2 - epsilon, P1, epsilon), (P4, P4, P3))
        
    The belief matrix has the following form:
        
        Π_belief = (( P1, 0.5*P2, 0.5*P2, 0), 
                    (0.5*(P2-epsilon), P1, 0.5*(P2-epsilon), epsilon), 
                    ( 0, 0, P1, 1 - P1), 
                    (P4, P4, 0, P3))
    """
    
    P1, P2, P3, P4 = Π[0][0], Π[0][1], Π[2][2], Π[2][0]
    epsilon = Π[1][2]
    
    Π_belief = (( P1, 0.5*P2, 0.5*P2, 0), 
                (0.5*(P2-epsilon), P1, 0.5*(P2-epsilon), epsilon), 
                ( 0, 0, P1, 1 - P1), 
                (P4, P4, 0, P3))
        
    # rewrite the real transition matrix
    Π_real = ((P1, 0.5*P2, 0.5*P2, 0),
              (P2-epsilon, 0.5*P1, 0.5*P1, epsilon), 
              (P2-epsilon, 0.5*P1, 0.5*P1, epsilon), 
              (P4, 0.5*P4, 0.5*P4, P3))
    
    # adjust state space
    z_vals = np.asarray(z_vals)
    z_vals = np.insert(z_vals, 1, z_vals[1])
    
    return np.asarray(Π_real), np.asarray(Π_belief), np.asarray(z_vals)

# write code with which you can compute a mean-preserving-spread
# for the 'perceived' state space

def exp_val(z_vals, P_z):
    """
    This function gives you the weighted average of all states of a 
    markov chain in the long run.
    
    Parameters
    -----------
    
    z_vals : state values of the markov chain
    P_z : transition matrix
    """
    mc = MarkovChain(P_z, state_values = z_vals) # get the stationary probs
    return np.sum(mc.stationary_distributions * z_vals) # compute weighted average

def mean_preserving_states(z_vals, Π_real, Π_belief, err_tol):
    exp_real = exp_val(z_vals=z_vals, P_z=Π_real)
    exp_belief = exp_val(z_vals=z_vals, P_z=Π_belief)
    z_vals_new = np.array(z_vals)
    diff = exp_real - exp_belief
    n = len(z_vals)
    
    s_dist = MarkovChain(Π_belief).stationary_distributions[0]
    w_high = s_dist[0]/(s_dist[0]+s_dist[-1]) #weight for high value state
    w_neg = 1 - w_high # weight for negatiive value state
    
    while abs(diff) > err_tol:
        if z_vals_new[-1] >= -0.3:
            z_vals_new[-1] = -0.3
            z_vals_new[0] = z_vals_new[0]+(w_high*diff)
            exp_real = exp_val(z_vals=z_vals, P_z=Π_real)
            exp_belief = exp_val(z_vals=z_vals_new, P_z=Π_belief)
            diff = exp_real - exp_belief
        else:
            z_vals_new[0], z_vals_new[-1] = z_vals_new[0]+(w_high*diff), z_vals_new[-1]+(w_neg*diff)
            exp_real = exp_val(z_vals=z_vals, P_z=Π_real)
            exp_belief = exp_val(z_vals=z_vals_new, P_z=Π_belief)
            diff = exp_real - exp_belief
            #print('{}, {}'.format(exp_real, exp_belief))
            
    print('r-shock state values: {}'.format(np.asarray(z_vals_new)))
    return np.asarray(z_vals_new)


def mean_and_var_preserving_states(z_vals, Π_real, Π_belief, err_tol, T=1000000):
    exp_real = exp_val(z_vals=z_vals, P_z=Π_real)
    exp_belief = exp_val(z_vals=z_vals, P_z=Π_belief)
    z_vals_new = np.array(z_vals)
    diff_exp = exp_real - exp_belief
    weights_exp = MarkovChain(Π_belief).stationary_distributions[0]
    weights_exp[2] = weights_exp[1]
    weights_exp = weights_exp/sum(weights_exp)
    n = len(z_vals)
    
    var_real = np.sqrt(np.var(MarkovChain(Π_real, z_vals).simulate(T)))
    var_belief = np.sqrt(np.var(MarkovChain(Π_belief, z_vals).simulate(T)))
    weights_var = MarkovChain(Π_belief).stationary_distributions[0][0:3]
    weights_var[2] = weights_var[1]
    weights_var = weights_var/sum(weights_var)
    diff_var = var_belief - var_real
    
    while abs(diff_exp) > err_tol or abs(diff_var) > err_tol:
    
        while abs(diff_exp) > err_tol:
            for i, weight in enumerate(weights_exp):
                z_vals_new[i] += weight*diff_exp
        
            exp_real = exp_val(z_vals=z_vals, P_z=Π_real)
            exp_belief = exp_val(z_vals=z_vals_new, P_z=Π_belief)
            diff_exp = exp_real - exp_belief
        
            var_real = np.sqrt(np.var(MarkovChain(Π_real, z_vals).simulate(T)))
            var_belief = np.sqrt(np.var(MarkovChain(Π_belief, z_vals_new).simulate(T)))
            diff_var = var_belief - var_real
    
        while abs(diff_var) > err_tol:
            for i, weight in enumerate(weights_var):
                if i == 0:
                    z_vals_new[i] -= weight*diff_var
                else:
                    z_vals_new[i] += weight*diff_var
                
            exp_real = exp_val(z_vals=z_vals, P_z=Π_real)
            exp_belief = exp_val(z_vals=z_vals_new, P_z=Π_belief)
            diff_exp = exp_real - exp_belief
        
            var_real = np.sqrt(np.var(MarkovChain(Π_real, z_vals).simulate(T)))
            var_belief = np.sqrt(np.var(MarkovChain(Π_belief, z_vals_new).simulate(T)))
            diff_var = var_belief - var_real
        
    print('means_err: {}'.format(diff_exp))
    print('variance_err: {}, real: {}, belief: {}'.format(diff_var, var_real, var_belief))
    print('r-shock state values: {}'.format(np.asarray(z_vals_new)))
    
    return np.asarray(z_vals_new)



In [140]:
P1, P3, epsilon = 0.6, 0.3, 0.005
z_vals = (1.5, 0.2, -0.5)

P_real, P_belief, z_vals = belief_decompose(trans_mat(P1,P3,epsilon), z_vals)

z_vals_new = mean_preserving_states(z_vals, 
                                    P_real, 
                                    P_belief, 
                                    1e-7)

# test whether the function works
exp_val(z_vals_new, P_belief), exp_val(z_vals, P_real)

r-shock state values: [ 2.78814597  0.2         0.2        -0.3       ]


(0.84315029357999483, 0.84315037929495773)

In [141]:
print('real matrix: \n \n{}'.format(P_real))
print('\n \n belief matrix: \n \n{}'.format(P_belief))
print('\n \n state space: \n \n{}'.format(z_vals))

real matrix: 
 
[[ 0.6    0.2    0.2    0.   ]
 [ 0.395  0.3    0.3    0.005]
 [ 0.395  0.3    0.3    0.005]
 [ 0.35   0.175  0.175  0.3  ]]

 
 belief matrix: 
 
[[ 0.6     0.2     0.2     0.    ]
 [ 0.1975  0.6     0.1975  0.005 ]
 [ 0.      0.      0.6     0.4   ]
 [ 0.35    0.35    0.      0.3   ]]

 
 state space: 
 
[ 1.5  0.2  0.2 -0.5]


When we are working with beliefs, we have to keep in mind that there are two kinds of 'realities' in this model.
<br\>
<br\>
One is the 'actual' reality where the 'real' values are realized by the 'real' transition matrix. The other one is the agent's 'belief-reality' where the agent makes decisions based on a belief-transition-matrix, he perceives the realizations differently than they are in the actual reality of the simulation of this model.
<br\>
<br\>
We are using a pessimistic belief-matrix. Consequently, in the agent's 'belief-reality', the realization of the interest rates will be much lower in average. Thus, the marginal utility of the assets will be much lower and then it's not so suprising that the agent is going to save less.
<br\>
<br\>
That's why I have written a code, which computes a mean-preserving-spread of the state values. When simulating the pessimistic case with a mean_preserving-spread, I should put the new state values only into the expactation of the agents as they are only the values of the states which are perceived by the agent and not the 'real' values.

In [149]:
# Then, I will write a class and corresponding functions, 
# which distinguish between the two different 
# kinds of transition matrices.

import numpy as np
from scipy.optimize import brentq
from quantecon import MarkovChain

class Pair:
    """
    This class, takes two independent markov processes, 
    combines their state spaces
    and gives the corresponding transition matrix.
    """    
    
    def __init__(self,
                 a_vals = [0, 1], # state values of the first mc
                 b_vals = [1, 0], # state values of the second mc
                 P_a = [[0.9, 0.1], [0.1, 0.9]],
                 P_b = [[0.8, 0.2], [0.2, 0.8]]):
        
        self.a_vals = np.asarray(a_vals)
        self.b_vals = np.asarray(b_vals)
        self.P_a = np.asarray(P_a)
        self.P_b = np.asarray(P_b)
        self.n = len(a_vals) * len(b_vals)
        
        # Build the array with combined state values
        self.s_vals = make_statespace(self.a_vals, self.b_vals)
        
        # Combine both transition matrices
        self.Q = np.kron(self.P_b, self.P_a)
    

def make_statespace(a, b):
    """
    returns state space s_vals, when two statespaces of two independend markov 
    chains a, b are combined to s:=(a_state, b_state)
    
    Parameters
    ----------
    a,b : state spaces in form of numpy arrays containing the state values of
          the two markov chains a and b.
          
    Returns
    ---------
    state space vector : Returns a two-dimentional array 
                         [[a_state1, b_state1], [a_state2, b_state1], ...etc]
    """
    
    s_vals = []

    for j in range(len(b)):
        for i in range(len(a)):
            s_vals.append([a[i], b[j]])
    return np.array(s_vals)

def fixed_point(K, c_init, err_tol=1e-3, max_iter=300):
    i, c = 0, K(c_init)
    if np.linalg.norm((c-c_init))<err_tol:
        return c
    while max_iter>i and np.linalg.norm((c-c_init))>=err_tol:
        i=i+1
        c_init=c
        c=K(c_init)
    return np.asarray(c)

class canersHousehold:
    
    def __init__(self, 
                 r=0.03,
                 w=1,
                 β=0.96, 
                 Π_w=((0.6, 0.4), (0.05, 0.95)),
                 Π_r_real=((0.6, 0.4, 0), (0.35, 0.6, 0.05), (0.35, 0.35, 0.3)),
                 Π_r_belief=((0.6, 0.4, 0), (0.3, 0.6, 0.1), (0.35, 0.35, 0.3)),
                 z_w_vals=(0.5, 1.0),
                 z_r_vals=(1.5, 0.2, -0.5),
                 b=0, 
                 grid_max=16, 
                 grid_size=50,
                 du=lambda x: 1/x,
                 mean_preserved = False, 
                 mean_and_var_preserved = False):
        
        # simplify names, set up array and change the input
        # in a way so that you can adjust the coleman operator
        # and the simulation of assets, 
        # belief trans. matrix -> in coleman operator
        
        self.du, self.w, self.r = du, w, r
        self.grid_max, self.grid_size = grid_max, grid_size
        self.Π_w, self.z_w_vals = Π_w, z_w_vals
        self.β, self.b = β, b
        self.asset_grid = np.linspace(-b, grid_max, grid_size)
        self.mean_preserved = mean_preserved
        self.mean_and_var_preserved = mean_and_var_preserved
        
        self.Π_r_belief = np.asarray(Π_r_belief) 
        
        self.Π_r_real = np.asarray(Π_r_real)
        
        self.z_r_vals = np.asarray(z_r_vals)
        
        # perceived values
        
        if mean_preserved: 
            self.z_r_vals_belief = mean_preserving_states(self.z_r_vals,
                                                       self.Π_r_real,
                                                       self.Π_r_belief,
                                                       1e-7)
        elif mean_and_var_preserved:
            self.z_r_vals_belief = mean_and_var_preserving_states(self.z_r_vals,
                                                               self.Π_r_real,
                                                               self.Π_r_belief,
                                                               1e-4)
        else:
            self.z_r_vals_belief = np.asarray(z_r_vals)
        
        # take the new transition matrices and combine them with
        # the transition matrix for the wage shock in order
        # to get Π_belief, Π_real and z_vals
        
        self.z_belief = Pair(a_vals=self.z_r_vals_belief, # use perceived values here
                                 b_vals=z_w_vals,
                                 P_a=self.Π_r_belief,
                                 P_b=Π_w)
        
        self.z_real = Pair(a_vals=self.z_r_vals,
                           b_vals=z_w_vals,
                           P_a=self.Π_r_real,
                           P_b=Π_w)
        
        # you need two different state spaces, one for the simulation of the 
        # markov chain which needs the real values and the other one
        # for the operator which needs the belief values
        
        self.z_vals, self.z_belief_vals = self.z_real.s_vals, self.z_belief.s_vals
        self.Π_belief, self.Π_real = self.z_belief.Q, self.z_real.Q


def belief_operator(c, cp):
    
    # === simplify names, set up arrays === #
    r, Π_belief, Π_r_belief, β, du, b, w = cp.r, cp.Π_belief, cp.Π_r_belief, cp.β, cp.du, cp.b, cp.w
    asset_grid, z_belief_vals, z_r_vals = cp.asset_grid, cp.z_belief_vals, cp.z_r_vals_belief
    z_w_vals = cp.z_w_vals
    z_size = len(z_belief_vals)
    vals = np.empty(z_size)
    
    def cf(a):
        for i in range(z_size):
            vals[i] = np.interp(a, asset_grid, c[:,i])
        return vals
    
    Kc = np.empty(c.shape)
    for i_a, a in enumerate(asset_grid):
        for i_z, z in enumerate(z_belief_vals):
            z_r, z_w = z[0], z[1]
            i_zr = i_z % len(z_r_vals) # get index for r-shock
            def h(t):
                expected_r = r*np.dot(z_r_vals, Π_r_belief[i_zr,:])
                expectation = np.dot(du(cf((1+r*z_r) * a + w * z_w - t)), Π_belief[i_z,:])
                return du(t) - max((1+expected_r) * β * expectation, du((1+r*z_r) * a + w * z_w + b))
            Kc[i_a, i_z] = brentq(h, 1e-8, (1+r*z_r) * a + w * z_w + b)
    return Kc

#    Kc = np.empty(c.shape)
#    for i_a, a in enumerate(asset_grid):
#        for i_z, z in enumerate(z_belief_vals):
#            z_r, z_w = z[0], z[1]
#            def h(t):
#                expectation = np.dot(np.multiply(1+r*np.tile(z_r_vals,len(z_w_vals)),du(cf((1+r*z_r) * a + w * z_w - t))), Π_belief[i_z,:])
#                return du(t) - max(β * expectation, du((1+r*z_r) * a + w * z_w + b))
#            Kc[i_a, i_z] = brentq(h, 1e-8, (1+r*z_r) * a + w * z_w + b)
#    return Kc


def initialize(cp):
    # === Simplify names, set up arrays === #
    r, b, w = cp.r, cp.b, cp.w
    asset_grid, z_vals = cp.asset_grid, cp.z_belief_vals
    shape = len(asset_grid), len(z_vals)
    c = np.empty(shape)
    
    for i_a, a in enumerate(asset_grid):
        for i_z, z in enumerate(z_vals):
                z_r, z_w = z[0], z[1]
                c_max = (1+r*z_r) * a + w * z_w + b
                c[i_a, i_z] = c_max
    return c

def pessimistic_asset_series(cp,
                             T=250000,
                             max_iter=150):
    """
    Simulates a time series of length T for assets, given optimal savings
    behavior and pessimistic Beliefs. Parameter cp is an instance of canersPessimist.
    """
    
    Π_real, z_vals, r, w = cp.Π_real, cp.z_vals, cp.r, cp.w
    mc = MarkovChain(Π_real)
    c_init = initialize(cp)
    K = lambda c: belief_operator(c, cp)
    c = fixed_point(K=K, c_init=c_init, max_iter=max_iter)
    cf = lambda a, i_z: np.interp(a, cp.asset_grid, c[:, i_z])
    a = np.zeros(T+1)
    interest_rates = np.zeros(T)
    wages = np.zeros(T)
    a[0]=1e-5
    z_seq = mc.simulate(T)
    for t in range(T):
        i_z = z_seq[t]
        a[t+1] = (1+r*z_vals[i_z][0]) * a[t] + w * z_vals[i_z][1] - cf(a[t], i_z)
        interest_rates[t] = r*z_vals[i_z][0]
        wages[t] = r*z_vals[i_z][1]
    return a, interest_rates, wages

In [150]:
cp = canersHousehold(mean_preserved=True)
cp.z_belief_vals

r-shock state values: [ 1.65188765  0.2        -0.47520202]


array([[ 1.65188765,  0.5       ],
       [ 0.2       ,  0.5       ],
       [-0.47520202,  0.5       ],
       [ 1.65188765,  1.        ],
       [ 0.2       ,  1.        ],
       [-0.47520202,  1.        ]])

In [151]:
# After that, I will simulate the asset choices given a interest rate, 
# compute the variance of the assets series and compare them.
P1, P3, epsilon, z_r_vals = 0.6, 0.3, 0.005, [1.5, 0.02, -0.5]
z_w_vals = (0.5, 1)
Π_r_real, Π_r_belief, z_r_vals = belief_decompose(trans_mat(P1, P3, epsilon),
                                              z_r_vals)
print('real matrix: \n \n{}'.format(Π_r_real))
print('\n \n belief matrix: \n \n{}'.format(Π_r_belief))
print('\n \n state space: \n \n{}'.format(z_r_vals))

real matrix: 
 
[[ 0.6    0.2    0.2    0.   ]
 [ 0.395  0.3    0.3    0.005]
 [ 0.395  0.3    0.3    0.005]
 [ 0.35   0.175  0.175  0.3  ]]

 
 belief matrix: 
 
[[ 0.6     0.2     0.2     0.    ]
 [ 0.1975  0.6     0.1975  0.005 ]
 [ 0.      0.      0.6     0.4   ]
 [ 0.35    0.35    0.      0.3   ]]

 
 state space: 
 
[ 1.5   0.02  0.02 -0.5 ]


In [152]:
# create instances for different cases
r=0.02
# 1. non-pessimistic
hh1 = canersHousehold(r=r,
                      z_r_vals=z_r_vals, 
                      z_w_vals=z_w_vals,  
                      Π_r_real=Π_r_real, 
                      Π_r_belief=Π_r_real)
# 2. pessimistic
hh2 = canersHousehold(r=r,
                      z_r_vals=z_r_vals, 
                      z_w_vals=z_w_vals, 
                      Π_r_real=Π_r_real, 
                      Π_r_belief=Π_r_belief)
# 3. pessimistic with mean-preserving-spread
hh3 = canersHousehold(r=r,
                      z_r_vals=z_r_vals, 
                      z_w_vals=z_w_vals,  
                      Π_r_real=Π_r_real, 
                      Π_r_belief=Π_r_belief, 
                      mean_preserved = True)
# 4. pessimistic with mean-preserving statespace and same variance
hh4 = canersHousehold(r=r,
                      z_r_vals=z_r_vals, 
                      z_w_vals=z_w_vals,  
                      Π_r_real=Π_r_real, 
                      Π_r_belief=Π_r_belief, 
                      mean_and_var_preserved = True)

#simulate the asset series
T=500000
max_iter=500
a_np, r_np, w_np = pessimistic_asset_series(hh1,
                                      T=T,
                                      max_iter=max_iter)
a_p, r_p, w_p = pessimistic_asset_series(hh2,
                                    T=T,
                                    max_iter=max_iter)
a_p_mp, r_p_mp, w_mp = pessimistic_asset_series(hh3,
                                          T=T,
                                          max_iter=max_iter)
a_p_mvp, r_p_mvp, w_mvp = pessimistic_asset_series(hh4,
                                          T=T,
                                          max_iter=max_iter)

r-shock state values: [ 2.82643065  0.02        0.02       -0.3       ]
means_err: 7.647226253448824e-05
variance_err: -8.284298811611635e-06, real: 0.7425169387921338, belief: 0.7425086544933222
r-shock state values: [ 1.86613625  0.48851389  0.48851389 -0.26009538]


In [153]:
hh4.z_belief_vals

array([[ 1.86613625,  0.5       ],
       [ 0.48851389,  0.5       ],
       [ 0.48851389,  0.5       ],
       [-0.26009538,  0.5       ],
       [ 1.86613625,  1.        ],
       [ 0.48851389,  1.        ],
       [ 0.48851389,  1.        ],
       [-0.26009538,  1.        ]])

In [155]:
std_wp, std_wnp, std_wpmp, std_wpmvp = np.sqrt(np.var(w_p)), np.sqrt(np.var(w_np)), np.sqrt(np.var(w_mp)), np.sqrt(np.var(w_mvp))
std_wp, std_wnp, std_wpmp, std_wpmvp

mean_wp, mean_wnp, mean_wpmp, mean_wpmvp = np.mean(w_p), np.mean(w_np), np.mean(w_mp), np.mean(w_mvp) 
mean_wp, mean_wnp, mean_wpmp, mean_wpmvp

(0.018897980000000002,
 0.018895679999999998,
 0.018900379999999994,
 0.018896799999999991)

In [170]:
std_ap, std_anp, std_apmp, std_apmvp = np.sqrt(np.var(a_p)), np.sqrt(np.var(a_np)), np.sqrt(np.var(a_p_mp)), np.sqrt(np.var(a_p_mvp))
std_ap, std_anp, std_apmp, std_apmvp

mean_ap, mean_anp, mean_apmp, mean_apmvp = np.mean(np.log(a_p)), np.mean(np.log(a_np)), np.mean(np.log(a_p_mp)), np.mean(np.log(a_p_mvp)) 
mean_ap, mean_anp, mean_apmp, mean_apmvp
std_ap, std_anp, std_apmp, std_apmvp

(0.051266585163587342,
 0.074160578273054736,
 0.084660908368135307,
 0.078806166149500395)

In [171]:
std_rp, std_rnp, std_rpmp, std_rpmvp = np.sqrt(np.var(r_p)), np.sqrt(np.var(r_np)), np.sqrt(np.var(r_p_mp)), np.sqrt(np.var(r_p_mvp))
std_rp, std_rnp, std_rpmp, std_rpmvp

mean_rp, mean_rnp, mean_rpmp, mean_rpmvp = np.mean(r_p), np.mean(r_np), np.mean(r_p_mp), np.mean(r_p_mvp) 
mean_rp, mean_rnp, mean_rpmp, mean_rpmvp

(0.015067043200000001,
 0.015114140800000004,
 0.015025974400000006,
 0.015041752000000004)

In [172]:
import pandas as pd

wages = [{'wages': 'non-pessisimistic assets', 'mean': mean_wnp, 'std': std_wnp},
         {'wages': 'pessisimistic assets',  'mean': mean_wp, 'std': std_wp},
         {'wages': 'pessisimistic assets, mean-preserved',  'mean': mean_wpmp,  'std': std_wpmp},
         {'wages': 'pessisimistic assets, mean-var-preserved',  'mean': mean_wpmvp,  'std': std_wpmvp}]
dfa = pd.DataFrame(wages)
dfa0 = dfa.set_index('wages')
dfa0

,mean,std
wages,,
non-pessisimistic assets,0.018896,0.003134
pessisimistic assets,0.018898,0.003131
"pessisimistic assets, mean-preserved",0.018900,0.003128
"pessisimistic assets, mean-var-preserved",0.018897,0.003133


In [173]:
assets = [{'assets': 'non-pessisimistic assets', 'mean': mean_anp, 'std': std_anp},
         {'assets': 'pessisimistic assets',  'mean': mean_ap, 'std': std_ap},
         {'assets': 'pessisimistic assets, mean-preserved',  'mean': mean_apmp,  'std': std_apmp},
         {'assets': 'pessisimistic assets, mean-var-preserved',  'mean': mean_apmvp,  'std': std_apmvp}]
dfa = pd.DataFrame(assets)
dfa0 = dfa.set_index('assets')
dfa0

,mean,std
assets,,
non-pessisimistic assets,-1.811708,0.074161
pessisimistic assets,-2.264532,0.051267
"pessisimistic assets, mean-preserved",-1.750965,0.084661
"pessisimistic assets, mean-var-preserved",-1.780783,0.078806


In [160]:
interest_rates = [{'interest-rates': 'non-pessisimistic case', 'mean': mean_rnp, 'std': std_rnp},
         {'interest-rates': 'pessisimistic case',  'mean': mean_rp, 'std': std_rp},
         {'interest-rates': 'pessisimistic case, mean-preserved',  'mean': mean_rpmp,  'std': std_rpmp},
         {'interest-rates': 'pessisimistic case, mean-var-preserved',  'mean': mean_rpmvp,  'std': std_rpmvp}]
dfi = pd.DataFrame(interest_rates)
dfi0 = dfi.set_index('interest-rates')
dfi0

,mean,std
interest-rates,,
non-pessisimistic case,0.015114,0.014849
pessisimistic case,0.015067,0.014850
"pessisimistic case, mean-preserved",0.015026,0.014848
"pessisimistic case, mean-var-preserved",0.015042,0.014851


In [161]:
from quantecon import MarkovChain
z_r_vals_mps = hh3.z_r_vals_belief
z_r_vals_mvp = hh4.z_r_vals_belief

# Let's simulate the interest rates how they are perceived by the agents in each case
T=1000000
r=0.02
# 1. case, non-pessimistic
perceived_mc_np = MarkovChain(Π_r_real, state_values=r*z_r_vals)
pr_np = perceived_mc_np.simulate(T)

# 2. case, pessimmistic
perceived_mc_p = MarkovChain(Π_r_belief, state_values=r*z_r_vals)
pr_p = perceived_mc_p.simulate(T)


# 3. case, pessimistic mean-presreved 
perceived_mc_pmp = MarkovChain(Π_r_belief, state_values=r*z_r_vals_mps)
pr_p_mp = perceived_mc_pmp.simulate(T)

# 3. case, pessimistic mean-and-var-presreved 
perceived_mc_pmvp = MarkovChain(Π_r_belief, state_values=r*z_r_vals_mvp)
pr_p_mvp = perceived_mc_pmvp.simulate(T)

In [162]:
pstd_rp, pstd_rnp, pstd_rpmp, pstd_rpmvp = np.sqrt(np.var(pr_p)), np.sqrt(np.var(pr_np)), np.sqrt(np.var(pr_p_mp)), np.sqrt(np.var(pr_p_mvp))
pstd_rp, pstd_rnp, pstd_rpmp, pstd_rpmvp

(0.014797556586411444,
 0.014849993789845885,
 0.025942314783962358,
 0.014829550513064282)

In [163]:
pmean_rp, pmean_rnp, pmean_rpmp, pmean_rpmvp = np.mean(pr_p), np.mean(pr_np), np.mean(pr_p_mp), np.mean(pr_p_mvp)
pmean_rp, pmean_rnp, pmean_rpmp, pmean_rpmvp

(0.0070091055999999992,
 0.015096518400000003,
 0.015079317044941535,
 0.015050403429147484)

In [164]:
perceived_interest_rates = [{'perceived interest-rates': 'non-pessisimistic case', 'mean': pmean_rnp, 'std': pstd_rnp},
         {'perceived interest-rates': 'pessisimistic case',  'mean': pmean_rp, 'std': pstd_rp},
         {'perceived interest-rates': 'pessisimistic case, mean-preserved',  'mean': pmean_rpmp,  'std': pstd_rpmp},
         {'perceived interest-rates': 'pessisimistic case, mean-var-preserved',  'mean': pmean_rpmvp,  'std': pstd_rpmvp}]
dfpi = pd.DataFrame(perceived_interest_rates)
dfpi0 = dfpi.set_index('perceived interest-rates')
dfpi0

,mean,std
perceived interest-rates,,
non-pessisimistic case,0.015097,0.014850
pessisimistic case,0.007009,0.014798
"pessisimistic case, mean-preserved",0.015079,0.025942
"pessisimistic case, mean-var-preserved",0.015050,0.014830


In [165]:
dfa0.to_latex('variances.tex')
dfi0.to_latex('variances2.tex')
dfpi0.to_latex('variances3.tex')

In [3]:
import numpy as np
np.linspace(1e-6, 0.025,10)

array([  1.00000000e-06,   2.77866667e-03,   5.55633333e-03,
         8.33400000e-03,   1.11116667e-02,   1.38893333e-02,
         1.66670000e-02,   1.94446667e-02,   2.22223333e-02,
         2.50000000e-02])